In [ ]:
import numpy as np
import pandas as pd
import os
from google.colab import drive

from skimage import io, metrics
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from scipy import stats, ndimage
from skimage.transform import resize
from PIL import Image

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
low_path = '/content/drive/MyDrive/Low Ultrasound Images'
high_path = '/content/drive/MyDrive/High Ultrasound Images'

In [ ]:
def resize_image(image):
  target_shape = (256, 256)
  return image.resize(target_shape, Image.Resampling.LANCZOS)

def load_and_resize_images_from_folder(folder):
  images = []
  for filename in os.listdir(folder):
    if filename.endswith('.tif'):
      img = Image.open(os.path.join(folder, filename)).convert('L')
      if img is not None:
        img_resized = resize_image(img)
        img_array = np.array(img_resized)
        images.append(img_array)
  return images

low_freq_images = load_and_resize_images_from_folder(low_path)
print(len(low_freq_images))
high_freq_images = load_and_resize_images_from_folder(high_path)
print(len(high_freq_images))

1000
1000


In [ ]:
def calculate_metrics(images):
  metrics_data = {
      'SNR': [],
      'PSNR': [],
      'CNR': [],
      'HCSR': [],
      'SSIM': [],
      'LCC': [],
      'SROCC': [],
      'MSE': [],
      'RMSE': [],
      'Err3': [],
      'Err4': [],
      'GAE': [],
      'Q': []
  }

  # Calculate the reference image using the median
  reference_img = np.array(Image.fromarray(np.uint8(np.median(images, axis=0))))
  reference_img_flat = reference_img.flatten()

  for img in images:
    img_flat = img.flatten()

    # Signal-to-Noise Ratio (SNR)
    signal = np.mean(img_flat)
    noise = np.std(img_flat)
    snr = signal / noise if noise != 0 else float('inf')

    # Peak Signal-to-Noise Ratio (PSNR)
    psnr = peak_signal_noise_ratio(reference_img, img)

    # Contrast-to-Noise Ratio (CNR)
    contrast = np.max(img_flat) - np.min(img_flat)
    cnr = contrast / noise if noise != 0 else float('inf')

    # High-Contrast Spatial Resolution (HCSR)
    edges = ndimage.sobel(img)
    hcsr = np.mean(edges) / np.mean(np.abs(img)) if np.mean(np.abs(img)) != 0 else float('inf')

    # Structural Similarity Index Measure (SSIM)
    ssim_val, _ = structural_similarity(img, reference_img, win_size=3, full=True)

    # Linear Correlation Coefficient (LCC)
    lcc = np.corrcoef(img_flat, reference_img_flat)[0, 1]

    # Spearman’s rank-order correlation coefficient (SROCC)
    srocc, _ = stats.spearmanr(img_flat, reference_img_flat)

    # Mean Square Error (MSE)
    mse = metrics.mean_squared_error(reference_img, img)

    # Root Mean Square Error (RMSE)
    rmse = np.sqrt(mse)

    # Normalized error summation (Err3 and Err4)
    err3 = np.mean(np.abs(reference_img - img) ** 3) ** (1/3)
    err4 = np.mean(np.abs(reference_img - img) ** 4) ** (1/4)

    # Geometric Average Error (GAE)
    gae = np.mean(np.log(np.abs(reference_img - img) + 1e-10))

    # Universal Quality Index (Q)
    mean_x = np.mean(reference_img)
    mean_y = np.mean(img)
    var_x = np.var(reference_img)
    var_y = np.var(img)
    cov_xy = np.cov(reference_img.flatten(), img.flatten())[0, 1]
    luminance = (2 * mean_x * mean_y + 1e-10) / (mean_x**2 + mean_y**2 + 1e-10)
    contrast = (2 * np.sqrt(var_x) * np.sqrt(var_y) + 1e-10) / (var_x + var_y + 1e-10)
    structure = (cov_xy + 1e-10) / (np.sqrt(var_x) * np.sqrt(var_y) + 1e-10)
    q = luminance * contrast * structure

    metrics_data['SNR'].append(snr)
    metrics_data['PSNR'].append(psnr)
    metrics_data['CNR'].append(cnr)
    metrics_data['HCSR'].append(hcsr)
    metrics_data['SSIM'].append(ssim_val)
    metrics_data['LCC'].append(lcc)
    metrics_data['SROCC'].append(srocc)
    metrics_data['MSE'].append(mse)
    metrics_data['RMSE'].append(rmse)
    metrics_data['Err3'].append(err3)
    metrics_data['Err4'].append(err4)
    metrics_data['GAE'].append(gae)
    metrics_data['Q'].append(q)

  return metrics_data

low_freq_metrics = calculate_metrics(low_freq_images)
high_freq_metrics = calculate_metrics(high_freq_images)

low_freq_df = pd.DataFrame(low_freq_metrics)
high_freq_df = pd.DataFrame(high_freq_metrics)

In [ ]:
# export result
low_freq_df.to_csv('/content/drive/My Drive/results log/low_freq_assessment.csv', index=False)
high_freq_df.to_csv('/content/drive/My Drive/results log/high_freq_assessment.csv', index=False)

In [ ]:
# t-tests between the two groups for each metric
for metric in low_freq_metrics.keys():
  t_stat, p_value = stats.ttest_ind(low_freq_df[metric], high_freq_df[metric])
  print(f"T-test for {metric}:")
  print(f"T-statistic: {t_stat:.2f}, P-value: {p_value:.3f}\n")

T-test for SNR:
T-statistic: 11.05, P-value: 0.000

T-test for PSNR:
T-statistic: 9.70, P-value: 0.000

T-test for CNR:
T-statistic: -10.79, P-value: 0.000

T-test for HCSR:
T-statistic: -15.18, P-value: 0.000

T-test for SSIM:
T-statistic: 9.76, P-value: 0.000

T-test for LCC:
T-statistic: 0.98, P-value: 0.329

T-test for SROCC:
T-statistic: 0.23, P-value: 0.822

T-test for MSE:
T-statistic: -9.39, P-value: 0.000

T-test for RMSE:
T-statistic: -9.64, P-value: 0.000

T-test for Err3:
T-statistic: 9.77, P-value: 0.000

T-test for Err4:
T-statistic: -11.00, P-value: 0.000

T-test for GAE:
T-statistic: -5.96, P-value: 0.000

T-test for Q:
T-statistic: 3.97, P-value: 0.000



In [ ]:
# Calculate average values for each metric
def display_average_metrics(df, group_name):
  print(f"Average values for each metric in {group_name}:")
  for metric in df.columns:
    average_value = df[metric].mean()
    print(f"{metric}: {average_value:.4f}")

# Display average values for Low-Frequency and High-Frequency Groups
display_average_metrics(low_freq_df, "Low-Frequency Group (Group A)")
display_average_metrics(high_freq_df, "High-Frequency Group (Group B)")


Average values for each metric in Low-Frequency Group (Group A):
SNR: 3.0075
PSNR: 18.3965
CNR: 7.3009
HCSR: 1.7030
SSIM: 0.4746
LCC: 0.3772
SROCC: 0.4012
MSE: 1027.6563
RMSE: 31.3367
Err3: 4.8166
Err4: 2.8380
GAE: 3.8347
Q: 0.2586
Average values for each metric in High-Frequency Group (Group B):
SNR: 2.5344
PSNR: 17.5902
CNR: 7.9319
HCSR: 2.6688
SSIM: 0.4268
LCC: 0.3647
SROCC: 0.3979
MSE: 1254.0300
RMSE: 34.5138
Err3: 4.8099
Err4: 2.8547
GAE: 4.0023
Q: 0.2250
